In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

c:\Users\admin\anaconda3\envs\voice\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load model and processor
processor = WhisperProcessor.from_pretrained("jakeyoo/whisper-medium-ja")
model = WhisperForConditionalGeneration.from_pretrained("jakeyoo/whisper-medium-ja")
model.config.forced_decoder_ids = None

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import pandas as pd
import librosa
#convert dataset Function library
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

In [4]:
import os.path

sentence = ["～さい","あなた","あのかた","あのひと","いしゃ","エンジニア","おいくつ",
            "かいしゃいん","がくせい","きょうし","ぎんこういん","けんきゅうしゃ","しゃいん",
            "せんせい","だいがく","だれ","でんき","どなた","なんさい","びょういん","みなさん",
            "わたし","わたしたち"]
df = {
    "path": [],
    "word": [],
}

for index in range(len(sentence)):
    for word in sentence:
        path = f"..\\audio\cliped_audio\A_class\A_class_audio_{index}\{word}.mp3"
        if(os.path.isfile(path)):
            df["path"].append(path)
            df["word"].append(word)
#print(len(df["path"]))
#print(len(df["word"]))

test_dataset = {"path":df["path"]}
test_dataset = pd.DataFrame(test_dataset)
#print(test_dataset)
test_dataset = Dataset(pa.Table.from_pandas(test_dataset))
#print(test_dataset["path"])

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["path"], sr=16_000)
    batch["array"] = speech_array
    #batch["sentence"] = batch["sentence"].upper()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
input_features = processor(test_dataset["array"], sampling_rate=16_000, return_tensors="pt").input_features 

Map: 100%|██████████| 225/225 [00:05<00:00, 42.60 examples/s]


In [5]:
# generate token ids
predicted_ids = model.generate(input_features)
predicted_ids

tensor([[50258, 50266, 50359,  ..., 50257, 50257, 50257],
        [50258, 50266, 50359,  ..., 50257, 50257, 50257],
        [50258, 50266, 50359,  ..., 50257, 50257, 50257],
        ...,
        [50258, 50266, 50359,  ..., 50257, 50257, 50257],
        [50258, 50266, 50359,  ..., 50257, 50257, 50257],
        [50258, 50266, 50359,  ..., 50257, 50257, 50257]])

In [6]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print("transcription:",transcription)

transcription: ['さい。', 'あなた。', 'あの方。', 'あの人。', '一冊。', 'エンジニア。', 'おいくつ。', '会社会', '学生。', '教師。', '原稿令。', '研究者。', '下位', '先生。', '大英学。', '誰？', '天気。', 'どうなった？', '何歳！', '', '皆さん、', 'わたし。', '私たち。', 'さい。', 'あなた。', 'あのカッター。', 'あの人。', '医者。', 'エンジニア', 'おいくつ？', '会社員。', '学生。', '教師。', '銀行印。', '研究者。', '下位', '先生。', '大学。', 'だめ。', '電気。', 'どなた？', '何歳！', '病院', '皆さん、', 'わたし。', '私たち。', 'したい', 'あなた。', 'あなた。', 'あの糸。', '医者。', 'エンジニア', 'おいくつ？', '会社員', '学生。', '教師。', '銀行印', '研究者', '社員。', '先生。', '大学', '誰？', '電気。', 'となった。', '何歳！', '病院。', '皆さん、', 'わたし。', '私たち。', 'さい。', 'あなた。', 'あの人。', '医者', 'エンジニア', 'おいくつ？', '会社員', '学生。', '教師。', '銀行印', '研究者', '社員。', '先生。', '内学', '誰？', '電気。', 'どうなった。', '何歳！', '病院', '皆さん、', 'わたし、', '私たち。', 'さい。', 'あなた。', 'あなた。', 'あの人。', '一箱', 'エンジニア', 'おいくつ？', '開箱飲', '学生。', '教師。', '丁酷癮', '研究者。', 'しゃいん', '先生。', '大学。', '誰？', '元気。', 'ほとんどになった。', 'なんさ。', '有音。', 'みなさん、', 'わたし。', '私たち。', 'さい。', 'あなた！', 'あの方？', 'あの人。', '一写。', 'エンジニア。', 'おいくつ？', '会社へ。', '学生。', '教師。', 'コウィン', '研究者', 'しゃい。', '先生。', '大学。', 'あれ？', 

In [7]:
sentence = ["さい。","あなた。","あの方。","あの人。","医者。",
            "エンジニア。","おいくつ。","会社員。","学生。","教師。",
            "ぎんこういん。","研究者。","しゃいん。","先生。","大学。",
            "誰。","電気。","どなた。","何歳！","病院。",
            "皆さん。","わたし。","私たち。","さい。","あなた。",
            "あの方。","あの人。","医者。","エンジニア。","おいくつ。",
            "会社員。","学生。","教師。","ぎんこういん。","研究者。",
            "しゃいん。","先生。","大学。","誰。","電気。",
            "どなた。","何歳！","病院。","皆さん。","わたし。",
            "私たち。","さい。","あなた。","あの方。","あの人。",
            "医者。","エンジニア。","おいくつ。","会社員。","学生。",
            "教師。","ぎんこういん。","研究者。","しゃいん。","先生。",
            "大学。","誰。","電気。","どなた。","何歳！",
            "病院。","皆さん。","わたし。","私たち。","さい。",
            "あなた。","あの人。","医者。","エンジニア。",
            "おいくつ。","会社員。","学生。","教師。","ぎんこういん。",
            "研究者。","しゃいん。","先生。","大学。","誰。",
            "電気。","どなた。","何歳！","病院。","皆さん。",
            "わたし。","私たち。","さい。","あなた。","あの方。",
            "あの人。","医者。","エンジニア。","おいくつ。","会社員。",
            #-----------------------
            "学生。","教師。","ぎんこういん。","研究者。","しゃいん。",
            "先生。","大学。","誰。","電気。","どなた。",
            "何歳！","病院。","皆さん。","わたし。","私たち。",
            "さい。","あなた。","あの方。","あの人。","医者。",
            "エンジニア。","おいくつ。","会社員。","学生。","教師。",
            "ぎんこういん。","研究者。","しゃいん。","先生。","大学。",
            "誰。","電気。","どなた。","何歳！","病院。",
            "皆さん。","わたし。","私たち。","さい。","あなた。",
            "あの方。","あの人。","医者。","エンジニア。","おいくつ。","会社員。",
            "学生。","教師。","ぎんこういん。","研究者。","しゃいん。",
            "先生。","大学。","誰。","電気。","どなた。",
            "何歳！","病院。","みなさん、","わたし。","私たち。",
            "さい。","あなた。","あの方。","あの人。","医者。",
            "エンジニア。","おいくつ。","会社員。","学生。","教師。",
            "ぎんこういん。","研究者。","しゃいん。","先生。","大学。",
            "誰。","電気。","どなた。","何歳！。","病院。",
            "皆さん、","わたし。","私たち。","さい。","あなた。",
            "あの方。","あの人。","医者。","エンジニア。","おいくつ。",
            "会社員。","学生。","教師。","ぎんこういん。","研究者。",
            "しゃいん。","先生。","大学。","誰。","電気。",
            #---------------------------
            "何歳！。","病院。","みなさん。","わたし。",
            "私たち。","さい。","あなた。","あの人。",
            "医者。","エンジニア。","会社員。","学生。",
            "教師。","ぎんこういん。","研究者。","しゃいん。","先生。",
            "大学。","誰。","電気。","何歳！",
            "病院。","みなさん。","わたし。","私たち。"]

In [8]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load

#librispeech_test_clean = load_dataset("librispeech_asr", "clean", split="test")

test_dataset = {"path":df["path"],"text":sentence}
test_dataset = pd.DataFrame(test_dataset)

test_dataset = Dataset(pa.Table.from_pandas(test_dataset))

processor = WhisperProcessor.from_pretrained("jakeyoo/whisper-medium-ja")
model = WhisperForConditionalGeneration.from_pretrained("jakeyoo/whisper-medium-ja").to("cuda")

def map_to_pred(batch):
    
    speech_array, sampling_rate = librosa.load(batch["path"], sr=16_000)
    batch["array"] = speech_array
    input_features = processor(batch["array"], sampling_rate=16_000, return_tensors="pt").input_features
    batch["reference"] = processor.tokenizer._normalize(batch['text'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

result = test_dataset.map(map_to_pred)

wer = load("wer")
print(100 * wer.compute(references=result["reference"], predictions=result["prediction"]))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map:   0%|          | 0/225 [00:00<?, ? examples/s]c:\Users\admin\anaconda3\envs\voice\lib\site-packages\transformers\models\whisper\tokenization_whisper.py:511: UserWarning: The private method `_normalize` is deprecated and will be removed in v5 of Transformers.You can normalize an input string using the Whisper English normalizer using the `normalize` method.
  warnings.warn(
c:\Users\admin\anaconda3\envs\voice\lib\site-packages\transformers\models\whisper\modeling_whisper.py:697: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Map: 100%|██████████| 225/225 [01:53<00:00,  1.98 examples/s]


29.333333333333332
